In [4]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('yc_startups_aggregated_by_company.csv')

# Display the first few rows of the DataFrame
print(df.head())

print(df.columns)

  web-scraper-order                            web-scraper-start-url  \
0   1725002158-8882  https://www.ycombinator.com/companies?batch=S24   
1   1725000592-7697  https://www.ycombinator.com/companies?batch=W23   
2   1724993555-2619  https://www.ycombinator.com/companies?batch=S17   
3    1724990276-368  https://www.ycombinator.com/companies?batch=S10   
4   1725002011-8762  https://www.ycombinator.com/companies?batch=S24   

                                           one-liner batch  \
0                    AI-powered patent due diligence   S24   
1             We help organizations go passwordless.   W23   
2  10by10 is a unique marketplace in the recruiti...   S17   
3                                    Memorial sites.   S10   
4  We make microbes enabling ultra cheap metal ex...   S24   

                 location         type             categories  \
0  San Francisco, CA, USA          B2B                  Legal   
1                     NaN          B2B               Security   

In [15]:
import pandas as pd
import psycopg2
import os
import json
from dotenv import load_dotenv

load_dotenv('.env.local')

# Retrieve the database URL from the environment variables
DATABASE_URL = os.getenv('POSTGRESS_DB')

# Update the database URL to use the correct format
DATABASE_URL = f"postgresql://{DATABASE_URL}"

# Connect to the PostgreSQL database
conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor()

# Create the table in the database
create_table_query = """
CREATE TABLE IF NOT EXISTS yc_startups (
    id SERIAL PRIMARY KEY,
    web_scraper_order VARCHAR,
    web_scraper_start_url VARCHAR,
    one_liner VARCHAR,
    batch VARCHAR,
    location VARCHAR,
    type VARCHAR,
    categories VARCHAR,
    profile_href VARCHAR,
    url_href VARCHAR,
    status VARCHAR,
    founded FLOAT,
    description TEXT,
    launches TEXT,
    company_linkedin_href VARCHAR,
    company_twitter_href VARCHAR,
    company_crunchbase_href VARCHAR,
    company_github VARCHAR,
    number_jobs INTEGER,
    team_size FLOAT,
    latest_news TEXT,
    latest_news_href VARCHAR,
    youtube_link VARCHAR,
    email VARCHAR,
    calendar_link VARCHAR,
    name VARCHAR,
    founder_count INTEGER,
    founders JSONB
);
"""
cur.execute(create_table_query)
conn.commit()

# Read the CSV file into a DataFrame
df = pd.read_csv('yc_startups_aggregated_by_company.csv')

# Iterate over the DataFrame and add each row to the database
for index, row in df.iterrows():
    founders = []
    for i in range(1, 9):
        if pd.notna(row[f'founder-{i}']):
            founders.append({
                'name': row[f'founder-{i}'],
                'linkedin': row[f'founder-{i}-linkedin-href'] if pd.notna(row[f'founder-{i}-linkedin-href']) else None,
                'twitter': row[f'founder-{i}-twitter-href'] if pd.notna(row[f'founder-{i}-twitter-href']) else None
            })
    
    try:
        insert_query = """
        INSERT INTO yc_startups (
            web_scraper_order, web_scraper_start_url, one_liner, batch, location, type, categories, profile_href, url_href, status, founded, description, launches, company_linkedin_href, company_twitter_href, company_crunchbase_href, company_github, number_jobs, team_size, latest_news, latest_news_href, youtube_link, email, calendar_link, name, founder_count, founders
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cur.execute(insert_query, (
            row['web-scraper-order'],
            row['web-scraper-start-url'],
            row['one-liner'],
            row['batch'],
            row['location'],
            row['type'],
            row['categories'],
            row['profile-href'],
            row['url-href'],
            row['status'],
            float(row['founded']) if pd.notna(row['founded']) else None,
            row['description'],
            row['launches'],
            row['company-linkedin-href'],
            row['company-twitter-href'],
            row['company-crunchbase-href'],
            row['company-github'],
            int(row['number-jobs']) if pd.notna(row['number-jobs']) else None,
            float(row['team-size']) if pd.notna(row['team-size']) else None,
            row['latest_news'],
            row['latest_news-href'],
            row['youtube_link'],
            row['email'],
            row['calendar_link'],
            row['name'],
            int(row['founder_count']) if pd.notna(row['founder_count']) else None,
            json.dumps(founders)
        ))
    except Exception as e:
        print(f"Error occurred while processing row {index}: {e}")
        conn.rollback()  # Rollback the transaction to avoid partial commits

# Commit the transaction to save the data to the database
try:
    conn.commit()
except Exception as e:
    conn.rollback()
    print(f"Error occurred: {e}")

# Close the cursor and connection
cur.close()
conn.close()